In [ ]:
import pandas as pd
import cx_Oracle
import datetime

### db

In [ ]:
def get_conn_ora():
    
    username = "DPKMAPP"
    password = "Kms12#$"
    host = "61.106.19.217"
    port = 33389
    database = "CDB1"
    dsn = f'{host}:{port}/{database}'
    encoding = 'UTF-8'

    # 오라클 클라이언트를 선언: windows 환경변수에 path를 잡으면 필요없음
    try:
        cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_12")
    except Exception as e:
        pass

    try:
        connection = cx_Oracle.connect(username, password, dsn, encoding=encoding)
    except Exception as ex:
        print('Could not connect to database:', ex)
        return ex

    print("SUCCESS: Connecting database succeeded")
    return connection

def get_kms_datadf(sql):
    try:
        connection = get_conn_ora()
     
        with connection.cursor() as cursor:
            cursor.execute(sql)
            rows = cursor.fetchall()
            cols = [x[0].lower() for x in cursor.description]
   
        df = pd.DataFrame(rows, columns = cols)
        for c in df.columns:
            if df[c].dtype == object:
                df[c] = df[c].astype("string")

        connection.close()

    except Exception as ex:
        err, = ex.args
        print('error code :', err.code)
        print('error message :', err.message)
        return None
   
    if len(df) ==0:
        print('No data found')  
        return None
    return df


In [ ]:
import plotly.express as px
df_inq['ym'] = df_inq['dt'].str[:6]
fig = px.line(df_inq, x="ym", y="inq_cnt", color='site')
fig.show()

In [ ]:
#컨텐츠 리스트
ctgr_cd ='CA'
temp = f" AND CTGR_ID =   '{ctgr_cd}'" if ctgr_cd !='CA' else  " AND CTGR_ID = 'CA' "
sql = f"""
        SELECT
            AA.CNTNT_ID        
            , AA.TITL 
            , (SELECT CD_NM FROM TBCTKC10 WHERE CD = AA.SAVE_STAT_CD AND GROUP_CD = 'SAVE_STAT_CD') AS CD_NM
            , AA.CTGR_ID
            , AA.CTGR_PATH
            , AA.REG_DTTM
            , AA.UPD_DTTM
            , ((SELECT COUNT(FILE_KEY) FROM TBCTKK08 WHERE CNTNT_ID = AA.CNTNT_ID AND USE_YN='Y') +
               (SELECT COUNT(FILE_KEY) FROM TBCTKK22 WHERE CNTNT_ID = AA.CNTNT_ID AND USE_YN='Y')) AS ATT_CNT
            , (SELECT count(*) FROM TBCGPT10 G WHERE G.CNTNT_ID = AA.CNTNT_ID ) AS EMB_CNT
            FROM (
            SELECT A.CNTNT_ID
                ,(CASE WHEN A.SAVE_STAT_CD = '00' THEN 
                            (SELECT 
                            (CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END) AS AUTH_YN
                            FROM TBCTKC21 A
                            INNER JOIN TBCTKC23 B
                                ON  A.ATRT_GROUP_ID = B.ATRT_GROUP_ID
                            WHERE A.ETC_INFO01 > '50') 
                    ELSE 'Y' END) AS VIEW_YN
                , A.TITL
                , A.CTGR_ID
                , A.REG_DTTM
                , A.UPD_DTTM
                , BB.CTGR_PATH
                , A.SAVE_STAT_CD
            FROM TBCTKK01 A
            INNER JOIN
                (  SELECT  CTGR_ID
                    , CTGR_NM
                    , CTGR_PATH
                    , CONNECT_BY_ISLEAF AS IS_LEAF
                    FROM TBCTKK12
                    START WITH USE_YN ='Y'
                {temp}
                CONNECT BY PRIOR CTGR_ID = HGRK_CTGR_ID AND USE_YN ='Y' ) BB
            ON  A.CTGR_ID = BB.CTGR_ID
            ) AA
        WHERE AA.VIEW_YN='Y'
        ORDER BY AA.CNTNT_ID desc
    """
df = get_kms_datadf(sql)
df['reg_dt'] = df['reg_dttm'].dt.date
df['site']   = df['ctgr_path'].apply(lambda x: x.split('>')[1])
df_a = df[df['cd_nm']!='삭제'].groupby(['reg_dt'])[['cntnt_id']].count().rename(columns={'cntnt_id':'reg_cnt'}).sort_index(ascending=False)

In [ ]:
from_dt = '20201219'
to_dt = '20240119'
#일자별 등록현황
sql = f"""
   SELECT  	ROW_NUMBER() OVER (ORDER BY A.INQ_CNT DESC)  AS ROW_NUMBER
    , A.CNTNT_ID
    , A.DT
	, A.TITL
	, A.INQ_CNT
	, A.CTGR_PATH
	, A.REG_DTTM
	, DPKMAPP.FNCTK_GET_USERNAME(A.REGR_ID, A.REGR_DEPT_CD, 'DEPT') AS REGR_NM
FROM (
	SELECT   
			 A.CNTNT_ID
		   , A.DT
		   , MAX(B.TITL)							AS TITL
		   , SUM(nvl(A.INQ_CNT, 0))					AS INQ_CNT
		   , MAX(B.CTGR_ID)							AS CTGR_ID
		   , MAX(C.CTGR_PATH) 						AS CTGR_PATH
		   , MAX(B.REG_DTTM) 						AS REG_DTTM
    	   , MAX(B.REGR_ID)							AS REGR_ID
    	   , MAX(B.REGR_DEPT_CD)					AS REGR_DEPT_CD
	  FROM DPKMAPP.TBCTKK03 A	   /* 컨텐츠 마스터  */
INNER JOIN DPKMAPP.TBCTKK01 B ON (A.CNTNT_ID = B.CNTNT_ID)							
  INNER JOIN (
            SELECT CTGR_ID
                 , CTGR_NM
                 , CTGR_PATH
            FROM DPKMAPP.TBCTKK12
            START WITH 1=1
			 AND CTGR_ID      ='CA'
            CONNECT BY PRIOR CTGR_ID = HGRK_CTGR_ID AND USE_YN ='Y'
       ) C ON (B.CTGR_ID = C.CTGR_ID)  /* 카테고리 */
	 WHERE 1=1
 	AND DT BETWEEN {from_dt} AND {to_dt}							
  GROUP BY A.CNTNT_ID, A.DT
     ) A
"""
df_inq = get_kms_datadf(sql)
df_inq['req_dt'] = df_inq['reg_dttm'].dt.date
df_inq = df_inq.groupby(['req_dt'])[['inq_cnt']].sum().sort_index(ascending=False)

### chart

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

show_cnt = 30
        # Create figure with secondary y-axis
fig.add_trace(
            go.Scatter(x=df_a.index[:show_cnt], y=df_a['reg_cnt'][:show_cnt], name="등록건수"), row=1, col=1,
            secondary_y=False,
)
fig.add_trace(
            go.Scatter(x=df_inq.index[:show_cnt], y=df_inq['inq_cnt'][:show_cnt], name="조회건수"), row=1, col=1,
            secondary_y=True,
)
fig.update_yaxes(title_text="등록건수")
fig.update_yaxes(title_text="등록건수2",secondary_y=True)

In [ ]:
dd = df_a.merge(df_inq, how='outer', left_index=True, right_index=True).fillna(0).sort_index(ascending=False)
# dd = dd.style.format({'reg_cnt': "{:.0f}",'inq_cnt': "{:.0f}"})

### loader

####  unstructed

In [ ]:
from unstructured.partition.pdf import partition_pdf
path = "static/"
''' 라이브러리문제로 실행 오류발생'''
# Extract images, tables, and chunk text
raw_pdf_elements = partition_pdf(
    filename= path +"test.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

In [ ]:
# !pip install opencv-python
!pip install numpy --upgrade

In [ ]:
!pip install --upgrade h5py

### PDFloader

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader

In [ ]:
file_name = 'static/test.pdf'
loader = PyPDFLoader(file_name)
documents = loader.load_and_split()

In [ ]:
documents

In [ ]:
# pip install pymupdf
loader = PyMuPDFLoader(file_name)
documents2 = loader.load_and_split()

In [ ]:
documents2

In [ ]:
# doc = fitz.open("static/test.pdf")
doc = fitz.open("static/chinese-table.pdf")

In [ ]:
# doc.get_toc()
doc.metadata
# doc.page_count
# doc.load_page(0)

In [ ]:
def show_image(item, title=""):
    """Display a pixmap.

    Just to display Pixmap image of "item" - ignore the man behind the curtain.

    Args:
        item: any PyMuPDF object having a "get_pixmap" method.
        title: a string to be used as image title

    Generates an RGB Pixmap from item using a constant DPI and using matplotlib
    to show it inline of the notebook.
    """
    DPI = 150  # use this resolution
    import numpy as np
    import matplotlib.pyplot as plt

    # %matplotlib inline
    pix = item.get_pixmap(dpi=DPI)
    img = np.ndarray([pix.h, pix.w, 3], dtype=np.uint8, buffer=pix.samples_mv)
    plt.figure(dpi=DPI)  # set the figure's DPI
    plt.title(title)  # set title of image
    _ = plt.imshow(img, extent=(0, pix.w * 72 / DPI, pix.h * 72 / DPI, 0))
    
import fitz
if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")
    
'''표 인식이 잘 안됨'''
# doc = fitz.open("static/test.pdf")
doc = fitz.open("static/chinese-table.pdf")
print(doc)
header = "Header"  # text in header
footer = "Page %i of %i"  # text in footer
for page in doc:
#     print(dir(page))
#     page.insert_text((50, 50), header)  # insert header
#     page.insert_text(  # insert footer 50 points above page bottom
#         (50, page.rect.height - 50),
#         footer % (page.number + 1, doc.page_count),
#     )
    print(page.get_text())


###### find table는 잘 인식되는듯하나 병합셀이 있으면 인식 불능으로 생각됨

In [ ]:
''' find tables '''
page = doc[0]
tabs = page.find_tables()  # detect the tables
for i,tab in enumerate(tabs):  # iterate over all tables
    for cell in tab.header.cells:
        page.draw_rect(cell,color=fitz.pdfcolor["red"],width=0.3)
    page.draw_rect(tab.bbox,color=fitz.pdfcolor["green"])
    print(f"Table {i} column names: {tab.header.names}, external: {tab.header.external}")
show_image(page, f"Table & Header BBoxes")

In [ ]:
page = doc[0]
tabs = page.find_tables(horizontal_strategy="text")
# print(len(tabs.tables))
print( f"{len(tabs.tables)} tables on {page}" )
tab = tabs[0]
for line in tab.extract():  # print cell text for each row
    print(line)
df = tab.to_pandas()

In [ ]:
# choose the second table for conversion to a DataFrame
tab = tabs[0]
df2 = tab.to_pandas()

### tablua

In [ ]:
import tabula

# PDF 파일로부터 테이블 추출
tables = tabula.read_pdf("static/test.pdf", pages="all")

# 추출된 테이블을 DataFrame으로 변환하여 사용
for tab in tables:
    print('----------')
    print(tab)
    

### html header splitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import HTMLHeaderTextSplitter
from langchain_community.document_loaders import BSHTMLLoader

url = "docs/여비규정.html"

headers_to_split_on = [  # 분할할 HTML 헤더 태그와 해당 헤더의 이름을 지정합니다.
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

# HTML 헤더를 기준으로 텍스트를 분할하는 HTMLHeaderTextSplitter 객체를 생성합니다.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# URL에서 텍스트를 가져와 HTML 헤더를 기준으로 분할합니다. 
#(split_text_from_url(),split_text_from_file(),split_text()
# html_header_splits = html_splitter.split_text_from_url(url)
# loader = BSHTMLLoader(url,'utf-8')
# documents = loader.load_and_split()
html_header_splits = html_splitter.split_text_from_file(url)

chunk_size = 900  # 텍스트를 분할할 청크의 크기를 지정합니다.
chunk_overlap = 100  # 분할된 청크 간의 중복되는 문자 수를 지정합니다.
text_splitter = RecursiveCharacterTextSplitter(  # 텍스트를 재귀적으로 분할하는 RecursiveCharacterTextSplitter 객체를 생성합니다.
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# HTML 헤더로 분할된 텍스트를 다시 청크 크기에 맞게 분할합니다.
splits = text_splitter.split_documents(html_header_splits)

# 분할된 텍스트 중 80번째부터 85번째까지의 청크를 출력합니다.
for header in splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

In [ ]:
for header in splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

### html tag간소화

In [1]:
''' 정규표현식을 활용한 tag제거 '''
# https://dbrang.tistory.com/1149
import re
# 함수선언
def subResult(regex, repl, text):
    value = re.sub(regex, repl, text)
#     if value == text:
#         print ('# -- '+ regex, '\n', value, '\n', '-' * 30)
    return value

def delete_tag(text,method="partial"):
    pattern = r'<style[^>]*>.*?</style>|style="[^"]*"'       # style= " " 태그 제거 
    value = subResult(pattern, "", text)
    if method=='all':
        value = subResult("<(/)?p[^>]*>", "", value)        # <p~></p~> 태그 제거
        value = subResult("<(/)?b[^>]*>", "", value)        # <b~></b~> 태그 제거
        value = subResult(r">\s+<", "><", value)            # 닫는 tag와 시작tag 사이의 공백제거
    else:
        value = subResult("<p[^>]*>", "", value)              # <p~> 태그 제거
    value = subResult("<(/)?span[^>]*>", " ", value)         # <spn~></span~> 태그 제거
    value = ' '.join(value.split())                          # 두개 이상의 공백을 하나로
    value = value.replace('&nbsp;','')
    return value

In [ ]:
f_name = '52156.html'
with open(f'docs/{f_name}','r', encoding='utf-8') as f:
    data = f.readlines()
    res = ''
    for text in data:
        value= delete_tag(text)
        res += value
print(res)

o_name = f_name.split('.')[0] + 'cnv.' + f_name.split('.')[1]
with open(f'docs/{o_name}','w', encoding='utf-8') as f:
            f.write(res)

In [ ]:
# res.replace('<br><br>','<br>')  
data

In [ ]:
import json

data = {"name":"alice"}
json_string = json.dumps(data)
print(json_string)

# with open("data.json","w") as f:
#     json.dump(data,f)

In [2]:
from pages import QA_chat as qc
from models import database as db
from models import html as htm
import json

C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
2024-05-02 17:18:49.617 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-02 17:18:49.636 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [11]:
def get_sql(cn_id):
    return f'''SELECT k01.cntnt_id AS cntnt_id 
             , k01.titl AS titl_nm
             , k13.cntnt_nm AS sec_nm
             , k13.cntnt_no as sec_id
             , k24.sub_cntnt_nm AS para_nm
             , k24.sub_cntnt_no as para_id
             , k24.contn AS contn
             , bb.ctgr_path
        FROM   tbctkk01 k01
             , tbctkk13 k13
             , tbctkk24 k24
             ,(  SELECT  CTGR_ID
                        , CTGR_NM
                        , CTGR_PATH
                        , CONNECT_BY_ISLEAF AS IS_LEAF
                        FROM TBCTKK12
                        START WITH USE_YN ='Y'
                       AND CTGR_ID = 'CA'
                CONNECT BY PRIOR CTGR_ID = HGRK_CTGR_ID AND USE_YN ='Y' ) bb
        where k01.cntnt_id = {cn_id}
          AND k01.cntnt_id = k13.cntnt_id
          AND k13.cntnt_id = k24.CNTNT_ID 
          AND k13.cntnt_no = k24.CNTNT_NO 
          AND k01.use_yn = 'Y'
          AND k13.use_yn = 'Y'
          AND k24.use_yn = 'Y'
          and k01.CTGR_ID = BB.CTGR_ID
        ORDER BY k13.sort_ord, k24.sort_ord'''

In [15]:
# cn_id = 52162
# # df = qc.get_kms_contents(cn_id)
# sql = get_sql(cn_id)
# df = db.get_kms_datadf(sql)

# '''간단한 방법이며 이 방법으로 일단 진행시킴'''
# dic = {}
# temp_dic = {}
# titl_nm,tab_nm,sub_nm  = ['', '', '']
# sec_i, para_i = [0, 0]

# for idx, row in df.iterrows():
#     sec_i = sec_i + 1 if row.tab_nm != tab_nm else sec_i
#     para_i = para_i + 1 if row.sub_nm != sub_nm else para_i

#     temp_dic['titl'] = row['titl_nm']
#     temp_dic['section'+str(sec_i)] = row.tab_nm
#     temp_dic['paragraph'+str(para_i)] = row.sub_nm
#     html_string = htm.make_header(row.contn, row.titl_nm)
#     html_string = delete_tag(html_string)
#     temp_dic['contents'+str(para_i)] = html_string
#     dic.update(temp_dic)
#     titl_nm, tab_nm, sub_nm = row.titl_nm,row.tab_nm,row.sub_nm
    
# metadata = f'id={df.cntnt_id[0]};path={df.ctgr_path[0]}'
# dd = {'data':dic, 'metadata': metadata}
# o_name = 'docs/' + str(cn_id) + '.json'
# with open(o_name, 'w', encoding='utf8') as f:
#      f.write(json.dumps(dd, ensure_ascii=False))
# print('succ')

SUCCESS: Connecting database succeeded
succ


In [33]:
cn_id = 52162
# df = qc.get_kms_contents(cn_id)
sql = get_sql(cn_id)
df = db.get_kms_datadf(sql)

'''공공사업실 방법 ==> 최종 24.5.2 '''
dic = {}
temp_dic = {}
sec_i, para_i = [0, 0]
titl_nm,sec_id,para_id  = ['', '', '']

lst = []
for idx, row in df.iterrows():
    sec_i = sec_i + 1 if row.sec_id != sec_id else sec_i
    para_i = 0 if row.sec_id != sec_id else  para_i  #텝이 바뀌면 순번 restart
    para_i = para_i + 1 if row.para_id != para_id else para_i
    temp_dic = {}
    temp_dic['title'] = row.titl_nm
    temp_dic['section'] = row.sec_nm
    temp_dic['paragraph'] = row.para_nm
#     html_string = htm.make_header(row.contn, row.titl_nm)
    html_string = row.contn
    html_string = delete_tag(html_string,"all")
    temp_dic['contents'] = html_string
    temp_dic['primary_key'] = f'{row.cntnt_id}>{row.sec_id}>{row.para_id}'
    temp_dic['path'] = row.ctgr_path
    lst.append(temp_dic)
    titl_nm, sec_id, para_id = row.titl_nm,row.sec_id,row.para_id

dd = dict({'data':lst})
o_name = 'docs/' + str(cn_id) + '.json'
with open(o_name, 'w', encoding='utf8') as f:
     f.write(json.dumps(dd, ensure_ascii=False))
print('succ')

SUCCESS: Connecting database succeeded
succ


In [34]:
dd

{'data': [{'title': '기본정책_쿠폰',
   'section': '쿠폰',
   'paragraph': '쿠폰',
   'contents': '1) 권한이 부여된 위블 비즈 관리자에 한해 아래 목적으로 쿠폰을 생성 할 수 있다. 가. 매출 활성화를 위한 이벤트 진행 나. 회원 및 서비스 정책 연계 혜택 지급(충전 리워드 쿠폰, Greener 레벨업 쿠폰 등) 다. 서비스 불만족시 고객 보상 라. 가격 조정이 필요한 대차 2) 생성된 쿠폰은 위블존/차량/상품/기간 등 관리자가 설정한 조건에 따라 발행되며, 회원은 쿠폰코드를 직접 입력하거나 쿠폰북에서 다운로드를 받음으로써 쿠폰을 발급받을 수 있다. 가. 회원 가입 마지막 단계에서 추천인 또는 추천코드를 입력하여 가입 완료시 맵핑된 쿠폰 발급 가능 나. 생성된 쿠폰을 앱내 내 쿠폰북에서 다운로드 가능 다. 생성된 쿠폰 코드를 앱내 내 쿠폰에서 입력하여 발급 가능 라. 서비스 이용 중 특정 시나리오에 조건 만족시 설정된 쿠폰 자동 발급  &#9312;  정회원 가입 완료 시 : 1만원 쿠폰 발급(개인 프로필에 한해 사용 가능)  &#9313;  가입단계에서 마케팅 전채널 수신 동의 시 : 5천원 쿠폰 자동 발급(개인 프로필에 한해 사용 가능)  &#9314;  친구 초대하여 정회원 가입 완료 시 : 추천인, 피추천인 모두 5천원 쿠폰 발급(개인 프로필에 한해 사용 가능), 무제한  &#9315;  Greener 등급 레벨업 시 : 별도 기술  &#9316;  충전 후 차량 반납 시 : 별도 기술 마. 관리자 시스템에서 회원에게 직접 쿠폰을 맵핑하여 발급 가능 3) 발행 가능기간이 지나면 쿠폰북에서 더이상 쿠폰을 발급 받을 수 없다. 4) 발급 받은 쿠폰은 내 쿠폰함으로 옮겨지며 해당 쿠폰의 유효기간(해당 날짜의 23:59까지)이 지나면 더이상 이용할 수 없다. 5) 모든 쿠폰은 결제시 대여요금에 한해 적용되며 대여건별 1개만 사용 가능하다. - 예외 #1. 1 Day+ 상품 구매시 할인율로만 쿠폰 적

In [5]:
cn_id = 52156
# df = qc.get_kms_contents(cn_id)
sql = get_sql(cn_id)
df = db.get_kms_datadf(sql)

'''공공사업실 방법 ==> 협의했던 방법(순번부여)'''
dic = {}
temp_dic = {}
titl_nm,sec_nm,para_nm  = ['', '', '']
sec_i, para_i = [0, 0]

lst = []
for idx, row in df.iterrows():
    sec_i = sec_i + 1 if row.sec_nm != sec_nm else sec_i
    para_i = 0 if row.sec_nm != sec_nm else  para_i  #텝이 바뀌면 순번 restart
    para_i = para_i + 1 if row.para_nm != para_nm else para_i
    temp_dic = {}
    temp_dic['section'+str(sec_i)] = row.sec_nm
    temp_dic['paragraph'+str(para_i)] = row.para_nm
#     html_string = htm.make_header(row.contn, row.titl_nm)
    html_string = row.contn
    html_string = delete_tag(html_string,"all")
    temp_dic['contents'+str(para_i)] = html_string
    lst.append(temp_dic)
    titl_nm, sec_nm, para_nm = row.titl_nm,row.sec_nm,row.para_nm
    
# metadata = f'title:{titl_nm},primary_key:{df.cntnt_id[0]},path={df.ctgr_path[0]}'
metadata = dict({'title':titl_nm,'primary_key':df.cntnt_id[0], 'path':df.ctgr_path[0]})
dd = dict({'data':lst, 'metadata':metadata})
o_name = 'docs/' + str(cn_id) + '.json'
with open(o_name, 'w', encoding='utf8') as f:
     f.write(json.dumps(dd, ensure_ascii=False))
print('succ')

SUCCESS: Connecting database succeeded
succ


In [ ]:
### 트리 구조형태이나 너무 복잡함 ###
dic = {}
titl_nm,sec_nm,para_nm  = ['', '', '']
sec_i, para_i = [0, 0]

for idx, row in df.iterrows():
#     print(row)
    sec_i = sec_i + 1 if row.sec_nm != sec_nm else sec_i
    print(sec_i, row.sec_nm)
    para_i = para_i + 1 if row.para_nm != para_nm else para_i

    dic['titl'] = row['titl_nm']
    dic['contents '+str(sec_i)] = {'title': row.sec_nm,
                      "para": {
                        'para'+str(para_i):
                            { 'title': row.para_nm,
                              'contents': "ss" #row.contn
                          }
                       }
                     }
    titl_nm, sec_nm, para_nm = row.titl_nm,row.sec_nm,row.para_nm

In [ ]:
df.groupby(level=-1).apply(lambda df: df.xs(df.name).to_dict()).to_dict()
# df.groupby(level=0)

In [ ]:
df.xs(df.name)

In [ ]:
for i in df.groupby(level=0):
    print(i)

In [ ]:
data_json4 = {
     "title": "제목명",
     "section": {
        "sec1" : {
            "title":"탭1명",
            "para": {
              "para1": { 
                "title" : "소제목1명",
                "contents": "내용1 html"
              },
              "para2": { 
                "title" : "소제목1명",
                "contents": "내용2 html"
             }
            }
          },

          "sec2" : {
            "title":"탭2명",
            "para": {
                "para1": { 
                    "title" : "소제목1명",
                    "contents": "내용1 html"
                },
                "para2": { 
                    "title" : "소제목1명",
                    "contents": "내용1 html"
                },
            }
          }
        }
      }

In [ ]:
data_json3

In [ ]:
data_json4

In [ ]:
data_json2